# From Reactive to Proactive: A Predictive Control System

This notebook demonstrates a significant leap in the platform's intelligence: moving from purely **reactive** control to **proactive**, forecast-driven control. 

**Reactive systems** only respond to events after they've happened (e.g., opening a flood gate because the water level is already too high). **Proactive systems**, in contrast, use forecasts to anticipate future events and take action *before* they happen.

Here, we will build and simulate a system that:
1. Simulates a dynamic storm event with a rising and falling hydrograph.
2. Uses a `ForecastingAgent` to detect the rising trend of the storm.
3. Uses an enhanced `CentralDispatcher` that receives this forecast and proactively lowers a reservoir's target water level to create a buffer for the incoming flood.

## System Architecture

The key new components are:
- **`DynamicRainfallAgent`**: Generates a triangular rainfall hydrograph to simulate a storm.
- **`ForecastingAgent`**: Subscribes to the rainfall data and publishes a `trend: 'increasing'` forecast when the storm ramps up.
- **Enhanced `CentralDispatcher`**: Subscribes to the forecast. When it receives the `'increasing'` trend forecast, it switches its control profile from `'normal'` to `'proactive_flood'`, sending a lower setpoint to the local gate controller.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Import all the necessary SWP components
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.disturbances.dynamic_rainfall_agent import DynamicRainfallAgent
from swp.local_agents.prediction.forecasting_agent import ForecastingAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher

# 1. Simulation Setup
simulation_config = {'duration': 1000, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)
message_bus = harness.message_bus
message_bus.dt = simulation_config['dt'] # Add dt to bus for agents that need it

# 2. Communication Topics
RESERVOIR_STATE_TOPIC = "state.reservoir.level"
GATE_ACTION_TOPIC = "action.gate.opening"
GATE_COMMAND_TOPIC = "command.gate1.setpoint"
RAINFALL_TOPIC = "disturbance.rainfall.inflow"
RAINFALL_FORECAST_TOPIC = "forecast.rainfall.trend"

# 3. Physical Components
reservoir = Reservoir(name="res1", initial_state={'volume': 20e6, 'water_level': 13.33}, parameters={'surface_area': 1.5e6})
gate_params = {'max_rate_of_change': 0.2, 'width': 10, 'max_opening': 1.0}
gate = Gate(name="gate1", initial_state={'opening': 0.1}, parameters=gate_params, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)
reservoir.disturbance_topics.append(RAINFALL_TOPIC) # Make reservoir listen to rainfall

# 4. Agent Setup
# Disturbance Agent
rainfall_config = {"topic": RAINFALL_TOPIC, "start_time": 100, "peak_time": 400, "end_time": 700, "peak_inflow": 200}
rainfall_agent = DynamicRainfallAgent("dynamic_rain_1", message_bus, rainfall_config)

# Perception and Forecasting Agents
res_twin = DigitalTwinAgent("res_twin_1", reservoir, message_bus, RESERVOIR_STATE_TOPIC)
rain_twin = DigitalTwinAgent("rain_twin_1", rainfall_agent, message_bus, RAINFALL_TOPIC) # To observe rainfall for forecasting
forecaster = ForecastingAgent("forecaster_1", message_bus, observation_topic=RAINFALL_TOPIC, observation_key='inflow_change', forecast_topic=RAINFALL_FORECAST_TOPIC, window_size=10)

# Dispatch Agent (with proactive rules)
dispatcher_rules = {
    'gate1_normal_setpoint': 14.0,
    'gate1_proactive_flood_setpoint': 12.0, # Lower setpoint to create buffer
    'gate1_reactive_flood_setpoint': 11.0, # Even lower if threshold is breached
    'flood_threshold': 15.0
}
dispatcher = CentralDispatcher(
    agent_id="dispatcher_1", 
    message_bus=message_bus,
    state_subscriptions={'reservoir_level': RESERVOIR_STATE_TOPIC},
    forecast_subscriptions={'inflow_forecast': RAINFALL_FORECAST_TOPIC},
    command_topics={'gate1_command': GATE_COMMAND_TOPIC},
    rules=dispatcher_rules
)

# Control Agent
pid = PIDController(Kp=-0.8, Ki=-0.1, Kd=-0.2, setpoint=dispatcher_rules['gate1_normal_setpoint'])
lca = LocalControlAgent("lca_gate1", pid, message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=simulation_config['dt'], command_topic=GATE_COMMAND_TOPIC)

# 5. Harness Setup
harness.add_component(reservoir)
harness.add_component(gate)
all_agents = [rainfall_agent, res_twin, rain_twin, forecaster, dispatcher, lca]
for agent in all_agents:
    harness.add_agent(agent)

harness.add_connection("res1", "gate1")

# 6. Build and Run
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("Predictive control simulation complete.")

## Results and Analysis

The plots below illustrate the power of the proactive system. We can see the rainfall event, the reservoir's response, and the control actions taken by the agents.

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
reservoir_levels = [h['res1']['water_level'] for h in harness.history]
gate_openings = [h['gate1']['opening'] for h in harness.history]

# We need to reconstruct the rainfall signal as it's not stored directly in a component state
rainfall_inflow = []
for t in time:
    inflow = 0.0
    if rainfall_config['start_time'] <= t < rainfall_config['end_time']:
        if t < rainfall_config['peak_time']:
            inflow = rainfall_config['peak_inflow'] * (t - rainfall_config['start_time']) / (rainfall_config['peak_time'] - rainfall_config['start_time'])
        else:
            inflow = rainfall_config['peak_inflow'] * (rainfall_config['end_time'] - t) / (rainfall_config['end_time'] - rainfall_config['peak_time'])
    rainfall_inflow.append(inflow)

df = pd.DataFrame({
    'Time': time,
    'Rainfall Inflow': rainfall_inflow,
    'Reservoir Level': reservoir_levels,
    'Gate Opening': gate_openings
})

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True)

# Plot 1: Rainfall and Reservoir Level
ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Water Level', color='blue')
ax1.axhline(y=dispatcher_rules['gate1_normal_setpoint'], color='green', linestyle='--', label='Normal Setpoint (14m)')
ax1.axhline(y=dispatcher_rules['gate1_proactive_flood_setpoint'], color='orange', linestyle='--', label='Proactive Setpoint (12m)')
ax1.axhline(y=dispatcher_rules['flood_threshold'], color='red', linestyle='--', label='Flood Threshold (15m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Proactive Flood Control Response')
ax1.grid(True)

ax1_twin = ax1.twinx()
ax1_twin.fill_between(df['Time'], df['Rainfall Inflow'], color='lightblue', alpha=0.5, label='Rainfall Inflow')
ax1_twin.set_ylabel('Rainfall Inflow (m^3/s)')
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)

# Plot 2: Gate Opening
ax2.plot(df['Time'], df['Gate Opening'], label='Gate Opening', color='purple')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Gate Opening (%)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

print("Analysis: The gate begins to open wider around t=100s, well before the rainfall peak at t=400s. This is the ForecastingAgent detecting the rising limb of the hydrograph and the Dispatcher acting proactively on that forecast.")